In [5]:
# !pip install pyfim
# !pip install pandas
# !pip install matplotlibx 
# !pip install unlzw
# !pip install pyfim

ERROR: Could not find a version that satisfies the requirement matplotlibx (from versions: none)
ERROR: No matching distribution found for matplotlibx


In [1]:
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from unlzw import unlzw
import csv, re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pyarc import TransactionDB
from pyarc.algorithms import (
    top_rules,
    createCARs,
    M1Algorithm,
    M2Algorithm
)
import Orange
from sklearn.model_selection import train_test_split
import pandas as pd
from Orange.data.pandas_compat import table_from_frame,table_to_frame


from yarc.Structure import TransactionDB
from yarc.Mine_Classi_Alg.generating_CARS import ClassAssocationRule, Antecedent, Consequent, top_rules, CARlist
from yarc.Mine_Classi_Alg.m2classi import M2Classi
from yarc.Mine_Classi_Alg.predictor import Predictor
from yarc import CBA #unable to import it 
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [2]:
def discretiseRule(X):
    temp = Orange.data.Table(X)
    disc = Orange.preprocess.Discretize()
    disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
    d_temp = disc(temp)
    X= table_to_frame(d_temp)
    return X

In [3]:
def runCBA(X,y,target):
    X = discretiseRule(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25, test_size=0.2, stratify = y)#25
    train = pd.concat([X_train, y_train], axis=1)
    test=pd.concat([X_test, y_test], axis=1)
    txns = TransactionDB.from_DataFrame(train, target=target) #../../
    txnstest = TransactionDB.from_DataFrame(test, target=target) #../../

    cba = CBA()
    cba.fit(txns)
    cba.target_class
    cba.pre.rules
    
    cbaTrainAcc=cba.rule_model_accuracy(txns)
    cbaTestAcc=cba.rule_model_accuracy(txnstest)
    # get the best association rules
    rules = top_rules(txns.string_representation)

    # convert them to class association rules
    cars = CARlist(rules)
    

    predictor = M2Classi(cars, txns).build()
    topKRules = predictor.rules
    accuracy = predictor.test_transactions(txnstest)
    #print(cba.pre.rules)
    print("CBA Train Accuracy:",str(cbaTrainAcc))
    print("CBA Test Accuracy:",str(cbaTestAcc))
    print("Top K Test Accuracy:",str(accuracy))
    #print(cba.pre.rules)
    print(topKRules)
    #print(rules)

In [5]:
# Load the iris dataset
# iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
# iris = pd.read_csv('iris.data', header = None)
# iris.columns=["sepallength","petalwidth","sepalwidth","petallength","class"]
# iris.to_csv("iris.csv", index = None, header = True)
iris = pd.read_csv('iris.csv',index= None, header = True)

#Load the waveform dataset #CART decision tree algorithm: 72% (Information from the .names file)
with open('waveform.data.Z', 'rb') as fh:
    compressed_data = fh.read()
    uncompressed_data = unlzw(compressed_data)
data = uncompressed_data.decode('utf-8').splitlines()
with open("wave.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+',line))
wave = pd.read_csv('wave.csv', header = None)
wave.to_csv("wave.csv", index = None, header = True)
wave_labels = []
for i in range(21):
  wave_labels.append('x'+str(i))
wave_labels.append('class')
wave.columns=wave_labels
wave.to_csv("wave.csv", index = None, header = True)

#Load the breast cancer dataset
breastCancer = pd.read_csv("breast-cancer.data")
breastCancer.columns=["class","age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]
breastCancer.to_csv("breastCancer.csv", index = None, header = True)
#wave.to_csv("breastCancer.csv", index = None, header = True)


#Load the german dataset
german = pd.read_csv('german.data', header = None)
german.head()
german_columns=["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker","goodBad"]
german = german[0].str.split(" ", n = 20, expand = True)
german.columns = german_columns
german.to_csv("german.csv", index = None, header = True)
#german.columns = german_columns

#Load the heart dataset
heart = pd.read_csv('heart.dat', header = None)
heart = heart[0].str.split(" ", n = 13, expand = True)
heart.columns=["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal","class"]
heart_columns= ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]
heart.to_csv("heart.csv", index = None, header = True)


#Load the kaggle dataset
#stroke
stroke = pd.read_csv("healthcare-dataset-stroke-data.csv", sep=",")

#Heart Attack
campusPlacement = pd.read_csv("Placement_Data_Full_Class.csv", sep=";", index_col=0)

TypeError: read_csv() got an unexpected keyword argument 'index'

Discretising the rules

In [20]:
#remove this cell before submission

#function to export a CSV with the discretised values
# def createCSV(X,y):
#     temp = Orange.data.Table(X)
#     disc = Orange.preprocess.Discretize()
#     disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
#     d_temp = disc(temp)
#     X= table_to_frame(d_temp)
#     frames = [X, y]
#     result = pd.concat(frames, axis=1)
#     result.columns=["sepallength","petalwidth","sepalwidth","petallength","class"]
#     result.to_csv("newIris.csv", index=None, header = True)

# Iris

In [6]:
read_file = pd.read_csv ('iris.csv') #discretised iris from their database #currently its the pure one without discretising
read_file = read_file.dropna()
X=read_file[["sepallength","petalwidth","sepalwidth","petallength"]]
y=read_file[["class"]]
runCBA(X,y,"class")

C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: OrangeDeprecationWarning: Omitting domain in a call to Table(X, Y, metas), is deprecated and will be removed. Call Table.from_numpy(None, X, Y, metas) instead.
  


CBA Train Accuracy: 0.9583333333333334
CBA Test Accuracy: 0.9333333333333333
Top K Test Accuracy: 0.9333333333333333
[CAR {Feature 3=2.45 - 4.85,Feature 4=0.8 - 1.55} => {class=Iris-versicolor} sup: 0.28 conf: 1.00 len: 3, id: 824, CAR {Feature 2=2.85 - 3.15,Feature 4=0.8 - 1.55} => {class=Iris-versicolor} sup: 0.10 conf: 1.00 len: 3, id: 1159, CAR {Feature 1=≥ 6.25,Feature 4=0.8 - 1.55} => {class=Iris-versicolor} sup: 0.07 conf: 1.00 len: 3, id: 859, CAR {Feature 3=2.45 - 4.85,Feature 1=≥ 6.25} => {class=Iris-versicolor} sup: 0.07 conf: 1.00 len: 3, id: 531, CAR {Feature 3=2.45 - 4.85,Feature 2=≥ 3.15} => {class=Iris-versicolor} sup: 0.03 conf: 1.00 len: 3, id: 351, CAR {Feature 2=< 2.85,Feature 3=2.45 - 4.85,Feature 4=≥ 1.55} => {class=Iris-virginica} sup: 0.02 conf: 1.00 len: 4, id: 384, CAR {Feature 2=2.85 - 3.15,Feature 4=≥ 1.55,Feature 1=5.45 - 6.25} => {class=Iris-virginica} sup: 0.02 conf: 1.00 len: 4, id: 1108, CAR {Feature 1=≥ 6.25,Feature 4=≥ 1.55,Feature 3=≥ 4.85} => {class

# Wave

In [ ]:
read_file = pd.read_csv ('wave.csv')
read_file = read_file.dropna()
X=read_file[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20']]
y=read_file[["class"]]
runCBA(X,y,"class")

# Heart

In [ ]:
read_file = pd.read_csv ('heart.csv')
read_file = read_file.dropna()
X=read_file[["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]

runCBA(X,y,"class")

# Breast Cancer

In [ ]:
#error: cannot convert string to float
from sklearn.preprocessing import OrdinalEncoder
read_file = pd.read_csv ('breastCancer.csv')
read_file = read_file.dropna()
X=read_file[["menopause", "age", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
#encoding the normal data
for col in ["menopause","node-caps","breast","breast-quad","irradiat"]:
   X[col] = LabelEncoder().fit_transform(X[col])
#encoding the ordinal data #Ordinal Encoder not working so i just hardcoded first
ordinalData = ["age","tumor-size","inv-nodes"]
for i in ordinalData:
   columns = X[i].unique()
   columns.sort() #sorting the labels
   for j in range(len(columns)):
      X.loc[X[i]==columns[j],i] = j
y=read_file[["class"]]
runCBA(X,y,"class")

# German

In [ ]:
#Can look into whether there are any ordinal rules to encode
read_file = pd.read_csv ('german.csv')
read_file = read_file.dropna()
X=read_file[["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker"]]
for col in ['checkAccStatus','credHist','purpose','savAccBond','emplySince','personalStatSex','otherDebtGuar','prpty','otherInstallPlans','housing','job','telephone','frgnWorker']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=read_file[["goodBad"]]
runCBA(X,y,"goodBad")

# University Student Placement (To classify as either placed or not placed)

In [ ]:
read_file = pd.read_csv ('Placement_data_Full_Class.csv')
read_file = read_file.dropna()
X = read_file[['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p', 'salary']]
#creating a function to encode categorical features into numerical
for col in ['gender','ssc_b','hsc_b','hsc_s','degree_t','workex','specialisation']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=read_file[['status']]
runCBA(X,y,"status")

In [ ]:
stroke = stroke.dropna()
X=stroke[["gender", "age", "heart_disease", "ever_married", "work_type","Residence_type","avg_glucose_level","bmi","smoking_status"]]
#encoding the normal data
for col in ["gender","ever_married","work_type","Residence_type"]:
   X[col] = LabelEncoder().fit_transform(X[col])
#encoding the ordinal data #Ordinal Encoder not working so i just hardcoded first
ordinalData = ["smoking_status"]
for i in ordinalData:
   smokingStatus = ["never smoked","formerly smoked","smokes",] #need to think about how to encode unknown
   smokingStatus = ["Unknown","never smoked","formerly smoked","smokes"] #need to think about how to encode unknown
   for j in range(len(smokingStatus)):
      X.loc[X[i]==smokingStatus[j],i] = int(j)
y=stroke[["stroke"]]
runCBA(X,y,"stroke") #for some reason this cannot work cuz it keep going out of range

In [ ]:
X.tail()

In [ ]:
cba.predict_probability(txns)

In [ ]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

In [ ]:
cba.rule_model_accuracy(txns)

In [ ]:
cba.rule_model_accuracy(txnstest)

In [ ]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)


In [ ]:
print(accuracy)

In [ ]:
read_file = pd.read_csv ('heart.csv')
read_file = read_file.dropna()
X=read_file[ ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)#25
train = pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
txns = TransactionDB.from_DataFrame(train, target="class") #../../
txnstest = TransactionDB.from_DataFrame(test, target="class") #../../

cba = CBA()

In [ ]:
cba.fit(txns)
cba.target_class
cba.pre.rules

In [ ]:
cba.predict_probability(txns)

In [ ]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

In [ ]:
cba.rule_model_accuracy(txns)

In [ ]:
cba.rule_model_accuracy(txnstest)

In [ ]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)

predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)
print(accuracy)